In [26]:
import time
from ore_algebra import *
DOP, t, D = DifferentialOperators()
# The following may be overridden in input file, but need not be specified there
#
#
defaultPrecision=1e-40;
# Do not forget to increase the precision of initial conditions if going beyond e-100
#
#
#
precision=[]; #if empty we will use the default precision for all systems
periodsOnly=true; #unless overridden 
paths=[]; #unless overridden we will convert this to a list of straight paths

#load("~/git-projects/periods/integrate/data/current.sage")
load("~/git-projects/period-suite/current.sage")
steps=len(allODEs);
cohomologies=[];
allTransMats=[1..steps];

if len(paths)==0:
    paths=[[0,1] for j in [1..steps]]
    
if len(precision)==0:
    precision=[defaultPrecision for j in [1..steps]]


t0=time.time();


print "Integrating systems", 1, "through", steps;
for j in [1..steps]:
    if periodsOnly and j==steps:
        noOfdeqs=geometricGenus;
    else:
        noOfdeqs=len(allODEs[j-1]);
    deqs=[deq.numerator() for deq in allODEs[j-1][0:noOfdeqs]];
    print "\nIntegrating system number", j;
    transitionMatricies = []
    for i in [1..noOfdeqs]:
        tt0=time.time()
        tm=deqs[i-1].numerical_transition_matrix(paths[j-1], precision[j-1],assume_analytic=true)
        print "\tODE number", i, "took", time.time()-tt0, "seconds to integrate"
        transitionMatricies=transitionMatricies+[tm]
    allTransMats[j-1]=transitionMatricies;
    print "Maximal error in transition matricies:"
    print [max(tm.apply_map(lambda x : x.diameter()).list()) for tm in transitionMatricies]
    print "The largest error:"
    print max([max(tm.apply_map(lambda x : x.diameter()).list()) for tm in transitionMatricies])
    cohomology=[1..noOfdeqs];
    for i in [1..noOfdeqs]:
        if j==1:
            init=Matrix(inits[i-1]);
            a = init.nrows(); b = init.ncols();
            init =  MatrixSpace(ComplexBallField(2000), a, b)(init);
        else:
            init=Matrix(rawInits[j-2][i-1])*cohomologies[j-2];
        transitionMatrix=transitionMatricies[i-1];
        cohomology[i-1]=(transitionMatrix * init).row(0);
    cohomologies=cohomologies+[Matrix(cohomology)];
    
periodsWithError=Matrix(cohomologies[-1].rows()[0:noOfdeqs]);
maximalError=max(periodsWithError.apply_map(lambda x : x.diameter()).list());
periods=periodsWithError.apply_map(lambda x : x.mid());
print "\nAccumulated maximal error:", maximalError
t1=time.time();
print "\nIt took", t1-t0, "seconds in total.\n"
print periods


Integrating systems 1 through 6

Integrating system number 1
	ODE number 1 took 0.346662998199 seconds to integrate
	ODE number 2 took 0.283285856247 seconds to integrate
	ODE number 3 took 0.280975103378 seconds to integrate
	ODE number 4 took 0.340201854706 seconds to integrate
	ODE number 5 took 0.628934144974 seconds to integrate
	ODE number 6 took 0.336512088776 seconds to integrate
	ODE number 7 took 0.603337049484 seconds to integrate
	ODE number 8 took 0.333140850067 seconds to integrate
	ODE number 9 took 0.292714118958 seconds to integrate
	ODE number 10 took 0.28659081459 seconds to integrate
	ODE number 11 took 0.336878061295 seconds to integrate
	ODE number 12 took 0.480561971664 seconds to integrate
	ODE number 13 took 0.271255016327 seconds to integrate
	ODE number 14 took 0.336494922638 seconds to integrate
	ODE number 15 took 0.647366046906 seconds to integrate
	ODE number 16 took 0.364181995392 seconds to integrate
	ODE number 17 took 0.682055950165 seconds to integra

In [17]:
#### Only for K3s
##
## Try a few values for "scale", do not exceed the negative exponent of accumulated error
##
scale = 80
##
## verify that you found honest relations by checking suspected relations using more digits
##
verify= 120
##
##

load("K3_intersection_matrix.sage")

periodVector=periods*intersectionMatrix.inverse()
u1 = periodVector.apply_map(real)[0].list()
u2 = periodVector.apply_map(imag)[0].list()
M = matrix([u1,u2])

proj = M.transpose()
Proj = (10^scale*proj).apply_map(lambda x : 10^3*x.round())
lattice = block_matrix([[Proj, matrix.identity(Proj.dimensions()[0])]])
reducedLattice = lattice.LLL()

Proj2 = (10^verify*proj).apply_map(lambda x : x.round())
testRelations = [v[2:23] * Proj2 for v in reducedLattice.rows()]

norms1 = [reducedLattice.row(i)[2:23].norm().n() for i in [0..20]]
norms2 = [testRelations[i].norm().n()+reducedLattice.row(i)[2:23].norm().n() for i in [0..20]]

norms=norms1

sortedNorms=sorted(norms)
consecutiveRatios = [sortedNorms[i]/sortedNorms[i-1] for i in [1..20]]

j=0; rank=1;
while j <= 19 :
    if consecutiveRatios[j] > 100 :
        j=20
    rank = rank + 1
    j=j+1
    
if sortedNorms[0] > 50 :
    rank = 1

print "\nPicard rank appears to be", rank, "\n"
print "Check to see if this makes sense:\n"

print "01","||", "---hyperplane section---"
j=0
while j <= 7 :
    print "0" + str(j+2), "||", norms[j], testRelations[j]
    j=j+1
while j <= 20 :
    print j+2, "||", norms[j], testRelations[j]
    j=j+1



Picard rank appears to be 18 

Check to see if this makes sense:

01 || ---hyperplane section---
02 || 1.41421356237310 (0, 0)
03 || 1.41421356237310 (0, 0)
04 || 1.41421356237310 (0, 0)
05 || 1.73205080756888 (1, 0)
06 || 1.73205080756888 (-1, 0)
07 || 1.73205080756888 (0, -1)
08 || 1.73205080756888 (0, 0)
09 || 1.73205080756888 (0, 0)
10 || 1.73205080756888 (0, 0)
11 || 1.73205080756888 (0, 0)
12 || 2.00000000000000 (0, 1)
13 || 2.00000000000000 (0, 1)
14 || 2.00000000000000 (0, 1)
15 || 2.00000000000000 (0, 0)
16 || 2.00000000000000 (0, 0)
17 || 1.73205080756888 (0, -1)
18 || 2.00000000000000 (0, 0)
19 || 1.08951748715406e30 (-122894134563875558929714804496, -194322660174680263357383986596)
20 || 2.87405578364192e30 (706467478964139949989645892690, -497748915515157207787029777583)
21 || 2.93899925438625e49 (-8006500986171464123145834214600452675722934511070, -119844993983469244123704678402789224558314358132647829225737536023451707567919800324414042)
22 || 4.55110908472018e49 (-1198

In [10]:
periods.nrows()

1